# Método BiCGSTAB (Bi-Conjugate Gradient Stabilized)
Cálculo Numérico - Sistemas Lineares

Alunos: Lara Hadasssa, Luiz Carlos, Rafael Nakayama


## Introdução
O método **BiCGSTAB (Bi-Conjugate Gradient Stabilized)** é um método iterativo usado para resolver sistemas lineares da forma:

\[Ax = b\]

Ele é indicado principalmente quando a matriz `A` é **grande, esparsa e não simétrica**.

- **Família**: métodos de Krylov.
- **Tipo**: iterativo.
- **Objetivo**: obter aproximação para `x` sem necessidade de fatorar a matriz.



## Estrutura
1. **Nome do método**: BiCGSTAB  
2. **Ideia central**: baseia-se no método Bi-Conjugate Gradient, mas com estabilização para melhorar a convergência.  
3. **Entrada**: matriz `A`, vetor `b`, chute inicial `x0`, tolerância `ε`, número máximo de iterações.  
4. **Saída**: solução aproximada `x`, resíduo, número de iterações.  


In [1]:

import numpy as np
from scipy.sparse.linalg import bicgstab

# Definindo matriz e vetor
A = np.array([[4, 1],
              [1, 3]])
b = np.array([1, 2])

# Resolvendo com BiCGSTAB
x, info = bicgstab(A, b)

print("Solução aproximada:", x)
print("Info:", info)  # 0 = sucesso


Solução aproximada: [0.09090909 0.63636364]
Info: 0


In [2]:

def bicgstab_custom(A, b, x0=None, tol=1e-8, max_iter=1000):
    n = len(b)
    if x0 is None:
        x = np.zeros(n)
    else:
        x = x0
    
    r = b - A @ x
    r_hat = r.copy()
    rho_old = alpha = omega = 1.0
    v = p = np.zeros(n)

    for k in range(1, max_iter+1):
        rho_new = np.dot(r_hat, r)
        if rho_new == 0:
            break
        if k == 1:
            p = r
        else:
            beta = (rho_new/rho_old) * (alpha/omega)
            p = r + beta * (p - omega * v)
        v = A @ p
        alpha = rho_new / np.dot(r_hat, v)
        s = r - alpha * v
        if np.linalg.norm(s) < tol:
            x += alpha * p
            print(f"Convergiu na iteração {k}")
            return x
        t = A @ s
        omega = np.dot(t, s) / np.dot(t, t)
        x += alpha * p + omega * s
        r = s - omega * t
        if np.linalg.norm(r) < tol:
            print(f"Convergiu na iteração {k}")
            return x
        rho_old = rho_new
    print("Não convergiu")
    return x

# Testando a implementação
A = np.array([[4, 1],
              [1, 3]])
b = np.array([1, 2])

x_custom = bicgstab_custom(A, b)
print("Solução customizada:", x_custom)


Convergiu na iteração 2
Solução customizada: [0.09090909 0.63636364]



## Características
- **Tipo**: Iterativo (família de Krylov).
- **Requisitos da matriz**:
  - Funciona para matrizes **não simétricas**.
  - Não exige ser definida positiva.
  - Pode ser usado com **pré-condicionadores**.



## Possíveis Problemas
1. Pode falhar para matrizes muito mal-condicionadas.
2. Em alguns casos não converge (ou converge muito lentamente).
3. É sensível à escolha de pré-condicionadores.



## Qualidades
- **Vantagens**:
  - Mais estável que BiCG.
  - Adequado para sistemas grandes e esparsos.
- **Melhor escolha quando**:
  - A matriz não é simétrica definida positiva (quando CG não serve).
  - O problema é muito grande para métodos diretos (LU, Cholesky).
